In [ ]:
if "google.colab" in str(get_ipython()):
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


In [ ]:
review_filename = 'data/yelp_academic_dataset_review.json'


In [ ]:
import pyspark
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
# sc = pyspark.SparkContext().getOrCreate()
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext


In [ ]:
raw_review_df = spark.read.json(review_filename)
# raw_review_df.printSchema()
raw_review_df.show()

raw_review_RDD = raw_review_df.rdd
# raw_review_RDD.first().asDict()


In [ ]:
def get_review_tuple(entry):
    """ Parse a line in the review dataset form pyspark.sql.Row to tuple
    Args:
        entry (pyspark.sql.Row): a line in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (str(entry["review_id"]),    # 0
            str(entry["user_id"]),      # 1
            str(entry["business_id"]),  # 2
            int(entry["stars"]),        # 3
            int(entry["useful"]),       # 4
            int(entry["funny"]),        # 5
            int(entry["cool"]),         # 6
            str(entry["text"]))         # 7


review_RDD = raw_review_RDD.map(get_review_tuple)  # repartition(2)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.take(3)}')


In [ ]:
#review_RDD.filter(lambda x: x[3]==0).count()


#stars - useful - funny - cool - numero fan


#if stars <3
    #stars - {[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2] + (numero fan/best user fans)}/2
#else
    #stars + {[useful + (funny + cool)/2] + (numero fan/best user fans)}

    #dati mancanti: numero fan; best fan; best useful/funny/cool per ristorante
#rdd_test_ufc = (id ristorsante, best useful, best funny, best cool)
#best fan = query su dataset utenti --> user_RDD.max().first()

In [ ]:
# review_best_ufc_RDD tuple: (business_id, (useful, funny, cool)): for each reastaurant, this tuple takes the maxima values of useful, funny and cool. 
review_best_ufc_RDD = review_RDD.map(lambda x : (x[2], (x[4], x[5], x[6]))).reduceByKey(lambda x, y : tuple(max(x[i], y[i]) for i in range(len(y))))

review_best_ufc_RDD.take(5)

In [ ]:
review_RDD.filter(lambda x: x[2]=='MWmXGQ98KbRo3vsS5nZhMA' and x[6]==3).take(20)